In [1]:
# 初始化
%load_ext autoreload
%autoreload 2
import sys
import os
os.chdir('E:\GitHub\QA-abstract-and-reasoning')
sys.path.append('E:\GitHub\QA-abstract-and-reasoning')

In [9]:
import tensorflow as tf

from seq2seq_tf2.model import Seq2Seq
from seq2seq_tf2.train_helper import train_model, get_train_msg
from seq2seq_tf2.batcher import batcher
from utils.config_gpu import config_gpu
from utils.params import get_params
from utils.saveLoader import Vocab
from utils.config import SEQ2SEQ_CKPT
import numpy as np

In [4]:
%run utils/params.py

In [5]:
# GPU资源配置
config_gpu()
# 读取vocab训练
vocab = Vocab(params["vocab_path"], params["vocab_size"])
params['vocab_size'] = vocab.count
# 构建模型
print("Building the model ...")
model = Seq2Seq(params)
# 获取保存管理者
checkpoint = tf.train.Checkpoint(Seq2Seq=model)
checkpoint_manager = tf.train.CheckpointManager(checkpoint, SEQ2SEQ_CKPT, max_to_keep=5)

checkpoint.restore(checkpoint_manager.latest_checkpoint)
if checkpoint_manager.latest_checkpoint:
    print("Restored from {}".format(checkpoint_manager.latest_checkpoint))
    params["trained_epoch"] = get_train_msg()
    params["learning_rate"] *= np.power(0.9, params["trained_epoch"])
else:
    print("Initializing from scratch.")
    params["trained_epoch"] = 0

1 Physical GPUs, 1 Logical GPUs
Building the model ...
Restored from E:\GitHub\QA-abstract-and-reasoning\data\checkpoints\seq2seq_checkpoints\ckpt-5


In [70]:
epochs = params['epochs']
batch_size = params['batch_size']
pad_index = vocab.word2id[vocab.PAD_TOKEN]
start_index = vocab.word2id[vocab.START_DECODING]
optimizer = tf.keras.optimizers.Adam(name='Adam', learning_rate=params["learning_rate"])
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

# 定义损失函数
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, pad_index))
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_sum(loss_) / tf.reduce_sum(mask)

def loss_function_old(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, pad_index))
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)

In [45]:
dataset = batcher(vocab, params)
steps_per_epoch =params["steps_per_epoch"]

In [46]:
enc_dec_inputs = next(iter(dataset))
enc_input = enc_dec_inputs["enc_input"]
dec_target = enc_dec_inputs["target"]

In [47]:
enc_output, enc_hidden = model.call_encoder(enc_input)

In [52]:
dec_input = tf.expand_dims([start_index] * batch_size, 1)

In [53]:
dec_hidden = enc_hidden

In [54]:
predictions, _ = model(dec_input, dec_hidden, enc_output, dec_target)

In [69]:
loss_function(dec_target[:, 1:], predictions)

<tf.Tensor: id=12585, shape=(), dtype=float32, numpy=4.1510983>

In [71]:
loss_function_old(dec_target[:, 1:], predictions)

<tf.Tensor: id=12627, shape=(), dtype=float32, numpy=2.223565>

损失函数

In [57]:
# input
real = dec_target[:, 1:]
pred = predictions

In [58]:
mask = tf.math.logical_not(tf.math.equal(real, pad_index))
loss_ = loss_object(real, pred)
mask = tf.cast(mask, dtype=loss_.dtype)
loss_ *= mask

In [63]:
tf.reduce_sum(loss_) / tf.reduce_sum(mask) 

<tf.Tensor: id=12529, shape=(), dtype=float32, numpy=4.1510983>

In [66]:
tf.reduce_sum(mask, axis=1)

<tf.Tensor: id=12535, shape=(64,), dtype=float32, numpy=
array([32.,  7., 12., 29., 23., 28., 20., 17., 17., 21., 10., 39., 14.,
       31., 18., 18., 30., 39.,  4.,  4., 27., 14., 14., 20., 27., 23.,
        6., 22., 18.,  9., 20., 14., 39., 17.,  6., 12., 27., 13., 11.,
       29., 13., 39., 14., 32., 28., 15., 14., 20., 30., 20., 13., 33.,
       16., 22., 33., 22., 20., 35., 20., 19., 12., 39., 15., 32.],
      dtype=float32)>

In [74]:
from utils.preprocess import get_seg_data

In [75]:
get_seg_data()

get_seg_data


E:\GitHub\QA-abstract-and-reasoning\utils\preprocess.py:415: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  _train_seg['train_seg_y'].to_csv(TRAIN_SEG_Y, index=None)
E:\GitHub\QA-abstract-and-reasoning\utils\preprocess.py:416: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  _test_seg['test_seg_x'].to_csv(TEST_SEG_X, index=None)
E:\GitHub\QA-abstract-and-reasoning\utils\preprocess.py:417: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  


create:  E:\GitHub\QA-abstract-and-reasoning\data\train_seg_x.csv
create:  E:\GitHub\QA-abstract-and-reasoning\data\train_seg_y.csv
create:  E:\GitHub\QA-abstract-and-reasoning\data\test_seg_x.csv
样本数量为: 81625
